<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [2]:
# cargar librerias 

# Estos paquetes los usaremos para analizar y visualizar los resultados
import pandas as pd
import tweepy
import re
import regex

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

from collections import Counter

import urllib.parse
import urllib.request
import time
import hmac
import hashlib
import random
import base64
import json

# Este es un modulo local de Python que contiene las credenciales de acceso para Twitter
import tweepy

In [3]:
# Leer llaves
keys = pd.read_csv("C:/Users/practicas/Desktop/Claves Tuit.csv", header=None)
keys= dict(zip(keys[0],keys[1]))

In [4]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'],keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True) # se autoregula para recolectarinformacion

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [5]:
ciudades = pd.read_csv("C:/Users/practicas/Documents/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [6]:
tweets = tweepy.Cursor(api.search,
                       q=['Seleccion colombia 2020 -RT'], # RT no incluir los retuis
                       lang='es', # solo tui en español
                           geocode=f'9.304744,-75.397778,45mi', #direccion geografica sincelejo cambia a la que se quiera
                       tweet_mode='extended', # para que no recorte los texto del tuit
                       include_rts=False).items(100) # limitar el numero de twitwes

In [7]:
t = []


for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Sincelejo")
    temp.append(tweet.user.location)
    temp.append(tweet.place)
    temp.append(tweet.coordinates)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)


In [8]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'place', 'coordinates', 'id', 'link']
data

,tweet,date,city,user_location,place,coordinates,id,link
0,Cereteano Alfredo Morelos de nuevo fue convoca...,2020-11-08 23:14:24,Sincelejo,"Montería, Colombia",None,None,1325577440547180545,https://twitter.com/i/web/status/1325577440547...
1,Eliminatorias: partidos de la selección Colomb...,2020-11-05 21:36:47,Sincelejo,"Montería, Colombia",None,None,1324465709796204551,https://twitter.com/i/web/status/1324465709796...
2,¡ATENCIÓN! #CORDOBA #OJOALDATO \nMontería estu...,2020-11-05 12:56:33,Sincelejo,"Córdoba, Colombia",None,None,1324334791051907074,https://twitter.com/i/web/status/1324334791051...


# Pre_ Prosesamiento twiters

In [9]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,Cereteano Alfredo Morelos de nuevo fue convoca...,2020-11-08 23:14:24,Sincelejo
1,Eliminatorias: partidos de la selección Colomb...,2020-11-05 21:36:47,Sincelejo
2,¡ATENCIÓN! #CORDOBA #OJOALDATO \nMontería estu...,2020-11-05 12:56:33,Sincelejo


quitar enlaces y menciones
quitar emojis

In [10]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [11]:
# nueva columna
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,tweet,date,city,clean,links,hashtags,mentions
0,Cereteano Alfredo Morelos de nuevo fue convoca...,2020-11-08 23:14:24,Sincelejo,Cereteano Alfredo Morelos de nuevo fue convoca...,"[https://t.co/UzoSL8GIcb, https://t.co/QkXUF7P...",[],"[@alfredomorelos0, @FAlfredomorelos]"
1,Eliminatorias: partidos de la selección Colomb...,2020-11-05 21:36:47,Sincelejo,Eliminatorias: partidos de la selección Colomb...,"[https://t.co/ou5uBA3Jeo, https://t.co/qSqWNaU...",[],[]
2,¡ATENCIÓN! #CORDOBA #OJOALDATO \nMontería estu...,2020-11-05 12:56:33,Sincelejo,¡ATENCIÓN! \nMontería estudia medidas transi...,[https://t.co/zT9rddZAr5],"[#CORDOBA, #OJOALDATO, #CHN]","[@SecGobMonteria, @PoliciaMonteria]"


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [12]:
# Libreias Plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [13]:
# contar por dias cuantos tuis hubieron 
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2020-11-05,2
1,2020-11-08,1


In [14]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

 
# Solo como ejemplo para ver dos lineas  se agrego 10 y colo rosado
trace2 = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values+10,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días Rosados',
                   line={'color':'pink'})

 

layout = go.Layout(title='Número de tuits por día')

 

fig = go.Figure(data=[trace, trace2], layout=layout)
iplot(fig)